In [16]:
from datasets import load_dataset
DATASET_NAME='qasc'
dataset = load_dataset("allenai/qasc", "", split=["train", "validation"])

In [17]:
dataset[0][0]

{'id': '3E7TUJ2EGCLQNOV1WEAJ2NN9ROPD9K',
 'question': 'What type of water formation is formed by clouds?',
 'choices': {'text': ['pearls',
   'streams',
   'shells',
   'diamonds',
   'rain',
   'beads',
   'cooled',
   'liquid'],
  'label': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']},
 'answerKey': 'F',
 'fact1': 'beads of water are formed by water vapor condensing',
 'fact2': 'Clouds are made of water vapor.',
 'combinedfact': 'Beads of water can be formed by clouds.',
 'formatted_question': 'What type of water formation is formed by clouds? (A) pearls (B) streams (C) shells (D) diamonds (E) rain (F) beads (G) cooled (H) liquid'}

In [18]:
def get_prompt(dataset, index, has_choice=False):
    try:
        json_line = dataset[index]
        question = json_line["question"]
        choices = json_line["choices"]
        choice_texts = choices["text"]
        perm=list(range(len(choice_texts)))
        choice_texts = [choice_texts[perm[i]].replace("\n", " ") for i in range(len(choice_texts))]
        candidates = " ".join(
            [
                f"({label if has_choice else ' '}) {text}"
                for text, label in zip(choice_texts, choices["label"])
            ]
        ).replace("\n", " ")
        answer_key = json_line["answerKey"]
        answer_key_idx = ord(answer_key[0]) - ord("A")
        answer_text = choice_texts[answer_key_idx]
        fact = f"{json_line['combinedfact']}{'' if json_line['combinedfact'][-1]=='.' else '.'}"
        prompt = f"{fact} {question} \\n {candidates}"
        return prompt, answer_text
    except:
        print('QUESTION: ',question,'ANSWER KEY', answer_key)

In [19]:
from tqdm import tqdm
import pickle
for index, dataname in enumerate(['train','test']):
    container=[]
    for ques_index, ques in enumerate(tqdm(dataset[index])):
        container.append(
                get_prompt(dataset[index], ques_index, has_choice=False)
            )
    pickle.dump(container, open(f"{DATASET_NAME}_{dataname}.pkl", "wb"))

100%|██████████| 926/926 [00:00<00:00, 4942.26it/s]
